In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import cvlib as cv

# Load the pre-trained gender detection model
model = load_model('gender_detection.model')

# Open webcam
webcam = cv2.VideoCapture(0)

# Check if webcam opened successfully
if not webcam.isOpened():
    print("Error: Could not access the webcam.")
    exit()

classes = ['Male', 'Female']

# Loop through frames
while True:
    # Read frame from webcam
    status, frame = webcam.read()

    if not status:
        print("Failed to grab frame.")
        break

    # Apply face detection
    faces, confidence = cv.detect_face(frame)

    # Check if faces are detected
    if faces is not None:
        # Loop through detected faces
        for idx, f in enumerate(faces):

            # Get corner points of face rectangle
            (startX, startY) = f[0], f[1]
            (endX, endY) = f[2], f[3]

            # Draw rectangle over face
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)

            # Crop the detected face region
            face_crop = np.copy(frame[startY:endY, startX:endX])

            if face_crop.shape[0] < 10 or face_crop.shape[1] < 10:
                continue  # Skip this face if it is too small

            # Preprocess face for gender detection model
            face_crop = cv2.resize(face_crop, (96, 96))
            face_crop = face_crop.astype("float") / 255.0
            face_crop = img_to_array(face_crop)
            face_crop = np.expand_dims(face_crop, axis=0)

            # Apply gender detection on the face
            conf = model.predict(face_crop)[0]

            # Get label with max confidence
            idx = np.argmax(conf)
            label = classes[idx]

            label = "{}: {:.2f}%".format(label, conf[idx] * 100)

            Y = startY - 10 if startY - 10 > 10 else startY + 10

            # Write label and confidence above face rectangle
            cv2.putText(frame, label, (startX, Y), cv2.FONT_HERSHEY_SIMPLEX,
                        0.7, (0, 255, 0), 2)

    # Display output
    cv2.imshow("Gender Detection", frame)

    # Press "Q" to stop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
webcam.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 23ms/step


1/1 [==============================] - 0s 17ms/step


1/1 [==============================] - 0s 51ms/step
